In [1]:
import xumm
import json
import asyncio

async def send_payment_request(amount, currency, source, destination, payment_reference):
    # Create the XUMM payment request payload
    xumm_payload = {
        "txjson": {
            "TransactionType": "Payment",
            "Account": source,
            "Destination": destination,
            "Amount" : str(amount),
        },
        "Fee":"12",
        "options": {
            "expire": 1800
        },
        "user_token":"xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx"
    }
    
    def callback_func(event):
        print('New payload event: {}'.format(event['data']))

        if 'signed' in event['data'] and event['data']['signed'] == True:

            print('The sign request was rejected :(')
            return event['data']

        if 'signed' in event['data'] and event['data']['signed'] == False:
            print('Woohoo! The sign request was signed :)')
            return False    

    # Create the payment request with the XUMM SDK
    xumm_client = xumm.XummSdk()
    subscription = await sdk.payload.create_and_subscribe(
      xumm_payload,
      callback_func,
    )

    # Print the payment request URL
    print(json.dumps(xumm_response.to_dict(), indent=4, sort_keys=True))
    print('New payload created, URL: {}'.format(subscription.created.next.always))
    print('  > Pushed: {}'.format('yes' if subscription.created.pushed else 'no'))

    """
    Now let's wait until the subscription resolved (by returning something)
    in the callback function.
    """
    resolve_data = await subscription.resolved()

    if resolve_data['signed'] == False:
        print('The sign request was rejected :(')

    if (resolve_data['signed'] == True):
        print('Woohoo! The sign request was signed :)')

    """
    Let's fetch the full payload end result, and get the issued
    user token, we can use to send our next payload per Push notification
    """
    result = sdk.payload.get(resolve_data['payload_uuidv4'])
    print('User token: {}'.format(result.application.issued_user_token))
    


In [3]:
loop = asyncio.get_event_loop()
loop.run_until_complete(send_payment_request(10000,'XRP','rPNJAfA5QnHTcZRduPqxAJAVcjiHRYjcsP','rDdZucpaMGFpvjqKRaHrqXjRg1PVStZSmh','test payment request'))

<coroutine object send_payment_request at 0x10ee8de40>

In [ ]:
import xrpl
import xrpl.wallet
from xumm import Xumm

def send_payment_transaction(amount, destination, sender_secret, sender_address):
    # Create a wallet object for the sender
    sender_wallet = xrpl.wallet.Wallet(seed=sender_secret, is_test=True)

    # Construct the payment transaction
    payment = xrpl.models.transactions.Payment(
        account=sender_wallet.classic_address,
        amount=str(amount),
        destination=destination,
        fee=xrpl.amounts.to_drops("0.000012"),
        sequence=xrpl.clients.TestnetClient().get_account_info(sender_wallet.classic_address).sequence
    )

    # Sign the transaction with the sender's wallet using the XRPL library
    signed_tx = xrpl.transaction.safe_sign_and_autofill_transaction(payment, sender_wallet)

    # Create the XUMM transaction payload
    xumm_payload = {
        "txjson": signed_tx.to_dict(),
        "options": {
            "submit": True
        }
    }

    # Sign the transaction with the XUMM wallet using the XUMM SDK
    xumm = Xumm(api_key='YOUR_API_KEY_HERE')
    xumm_response = xumm.create_payload(xumm_payload)
    xumm_signed_tx = xumm.get_signed_tx_hex(xumm_response.payload_uuid, sender_wallet.seed)

    # Submit the signed transaction to the XRPL testnet
    client = xrpl.clients.TestnetClient()
    response = client.submit(xumm_signed_tx)
    print(response)


In [ ]:
{
    "next": {
        "always": "https://xumm.app/sign/3bbba11b-7481-4994-b862-557dc3198460"
    },
    "pushed": false,
    "refs": {
        "qr_matrix": "https://xumm.app/sign/3bbba11b-7481-4994-b862-557dc3198460_q.json",
        "qr_png": "https://xumm.app/sign/3bbba11b-7481-4994-b862-557dc3198460_q.png",
        "qr_uri_quality_opts": [
            "m",
            "q",
            "h"
        ],
        "websocket_status": "wss://xumm.app/sign/3bbba11b-7481-4994-b862-557dc3198460"
    },
    "uuid": "3bbba11b-7481-4994-b862-557dc3198460"
}